In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [0]:
df_prod = spark.read.table("my_database.my_schema.ext_smc_prod")
df_serv = spark.read.table("my_database.my_schema.service_offerings")

In [0]:
total_columns = df_serv.columns
for i in range(len(total_columns)):
    df_serv = df_serv.withColumnRenamed(total_columns[i],
                                  'SOF_' + total_columns[i])

In [0]:
joined_data = df_prod.join(df_serv,df_prod.CONTRACT_NUMBER == df_serv.SOF_CONTRACT_NUMBER, how="left")

In [0]:
df_a = joined_data.filter(col("SOF_Service_Offering_Stand-by") == "Y")
df_b = joined_data.filter(col("SOF_Service_Offering_365j") == "Y")
df_c = joined_data.filter(col("SOF_Service_Offering_Preventive_Maintenance_hours") == "Y")
df_d = joined_data.filter(col("SOF_Service_Offering_Break-fix_hours") == "Y")
df_e = joined_data.filter(col("SOF_Service_Offering_Sub-contractor_labour") == "Y")
df_f = joined_data.filter(col("SOF_Service_Offering_Parts") == "Y")
df_g = joined_data.filter(col("SOF_Service_Offering_Special_agreement") == "Y")
df_h = joined_data.filter(col("SOF_Service_Offering_Software") == "Y")

In [0]:
df_a = df_a.withColumn("Temp_Serv_Offier_Details",lit("Service_Offering_Stand-by"))\
            .withColumn("INVENTORY_ITEM_ID",lit("900000003"))\
            .withColumn("SMC_Coverage",lit("lkpSO_Coverage_Template_Stand_by"))\
            .withColumn("seq_no",lit("1"))

In [0]:
df_b = df_b.withColumn("Temp_Serv_Offier_Details",lit("Service_Offering_365j"))\
            .withColumn("INVENTORY_ITEM_ID",lit("900000001"))\
            .withColumn("SMC_Coverage",lit("lkpSO_Coverage_Template_365j"))\
            .withColumn("seq_no",lit("2"))

In [0]:
df_c = df_c.withColumn("Temp_Serv_Offier_Details",lit("Service_Offering_Preventive_Maintenance_hours"))\
            .withColumn("INVENTORY_ITEM_ID",lit("900000000"))\
            .withColumn("SMC_Coverage",lit("lkpSO_Coverage_Template_Prev__Maintenance_hours"))\
            .withColumn("seq_no",lit("3"))

In [0]:
df_d = df_d.withColumn("Temp_Serv_Offier_Details",lit("Service_Offering_Break-fix_hours"))\
            .withColumn("INVENTORY_ITEM_ID",lit("900000001"))\
            .withColumn("SMC_Coverage",lit("lkpSO_Coverage_Template_Break_fix_hours"))\
            .withColumn("seq_no",lit("4"))

In [0]:
df_e = df_e.withColumn("Temp_Serv_Offier_Details",lit("Service_Offering_Sub-contractor_labour"))\
            .withColumn("INVENTORY_ITEM_ID",lit("900000005"))\
            .withColumn("SMC_Coverage",lit("lkpSO_Coverage_Template_Sub_contractor_labour"))\
            .withColumn("seq_no",lit("5"))

In [0]:
df_f = df_f.withColumn("Temp_Serv_Offier_Details",lit("Service_Offering_Parts"))\
            .withColumn("INVENTORY_ITEM_ID",lit("900000002"))\
            .withColumn("SMC_Coverage",lit("lkpSO_Coverage_Template_Parts"))\
            .withColumn("seq_no",lit("6"))

In [0]:
df_g = df_g.withColumn("Temp_Serv_Offier_Details",lit("Service_Offering_Special_agreement"))\
            .withColumn("INVENTORY_ITEM_ID",lit("900000008"))\
            .withColumn("SMC_Coverage",lit("lkpSO_Coverage_Template_Special_agreement"))\
            .withColumn("seq_no",lit("7"))

In [0]:
df_h = df_h.withColumn("Temp_Serv_Offier_Details",lit("Service_Offering_Software"))\
            .withColumn("INVENTORY_ITEM_ID",lit("900000004"))\
            .withColumn("SMC_Coverage",lit("lkpSO_Coverage_Template_Software"))\
            .withColumn("seq_no",lit("8"))

In [0]:

df_union = df_a.union(df_b).union(df_c).union(df_d).union(df_e).union(df_f).union(df_g).union(df_h)

In [0]:
df_sorted = df_union.sort(asc("CONTRACT_NUMBER"), asc("seq_no"))

In [0]:
df_window = Window.partitionBy("CONTRACT_NUMBER").orderBy('seq_no')
df_with_seq = df_sorted.withColumn('Linenumber', row_number().over(df_window))


In [0]:
df_ser_desc= spark.read.table("my_database.my_schema.service_offering_desc")
df_ser_detail= spark.read.table("my_database.my_schema.service_offering_details")

In [0]:
df_ser_desc=df_ser_desc.dropDuplicates(['Item'])


In [0]:

df_join_desc = df_with_seq.join(df_ser_desc,df_with_seq.INVENTORY_ITEM_ID == df_ser_desc.Item , how="left")


In [0]:
df_join_final = df_join_desc.join(df_ser_detail,df_join_desc.Temp_Serv_Offier_Details == df_ser_detail.Legacy_service_offering , how="left")

In [0]:
df_join_final=df_join_final.withColumn("XX_OSS_ServiceOfferingDescription_c",col("Description")).withColumn("XX_OSS_Service_Offering_Details_c",col("SMC_Service_Offering_Details")).withColumn("SRC_ORIGINAL_PRICE",col("ORIGINAL_PRICE"))

In [0]:
required_columns = ["Linenumber","XX_OSS_ServiceOfferingDescription_c","INVENTORY_ITEM_ID","SMC_Coverage","CONTRACT_NUMBER","NAME","ORG_ID","ORIGINAL_PRICE","BILL_TO_SITE_USE_ID","END_DATE","EXTN_ATTRIBUTE_CHAR003","BILL_TO_ACCT_ID","SERVICE_OFFERINGS","EXT_INVENTORY_ITEM_ID","DFF1_S_BILLING_TIMING","PROJECT_STATUS_CODE","JCI_BRANCH_NUMBER","OUTPUT_TAX_CLASSIFICATION","DFF1_S_BILLING_METHOD","SRC_ORIGINAL_PRICE","XX_OSS_Service_Offering_Details_c"
]  # Replace with your actual FBDI template order

# Reorder/select columns
df_join_final = df_join_final.select(required_columns)

In [0]:
df_join_final.write.mode('overwrite').option("mergeSchema","true").saveAsTable("my_database.my_schema.Tgt_SMC_integarte_int")